# How to gauge-optimize to a gate set other than the ideal targets 
Typically gauge optimizations are performed with respect to the set of ideal target gates and spam operations.  This is convenient, since you need to specify the ideal targets as points of comparison, but not always the best approach.  Particularly when you expect all or some of the gate estimates to either substantially differ from the ideal operations or differ, even by small amounts, in particular ways from the ideal operations, it can be hugely aid later interpretation to specify a non-ideal `GateSet` as the target for gauge-optimization.  By separating the "ideal targets" from the "gauge optimization targets", you're able to tell the gauge optimizer what gates you *think* you have, including any known errors.  This can result in a gauge-optimized estimate which is much more sensible and straightforward to interpet.

For example, gauge transformations can slosh error between the SPAM operations and the non-unital parts of gates.  If you know your gates are slightly non-unital you can include this information in the gauge-optimization-target (by specifying a `GateSet` which is slightly non-unital) and obtain a resulting estimate of low SPAM-error and slightly non-unital gates.  If you just used the ideal (unital) target gates, the gauge-optimizer, which is often setup to care more about matching gate than SPAM ops, could have sloshed all the error into the SPAM ops, resulting in a confusing estimate that indicates perfectly unital gates and horrible SPAM operations.

This example demonstrates how to separately specify the gauge-optimization-target `GateSet`.  There are two places where you might want to do this: 1) when calling `pygsti.do_long_sequence_gst`, to direct the gauge-optimization it performs, or 2) when calling `estimate.add_gaugeoptimized` to add a gauge-optimized version of an estimate after the main GST algorithms have been run.  

In both cases, a dictionary of gauge-optimization "parameters" (really just a dictionary of arguments for `pygsti.gaugeopt_to_target`) is required, and one simply needs to set the `targetGateset` argument of `pygsti.gaugeopt_to_target` by specifying `targetGateset` within the parameter dictionary.  We demonstrate this below.

First, we'll setup a standard GST analysis as usual except we'll create a `gs_guess` gate set that is meant to be an educated guess at what we expect the estimate to be.  We'll gauge optimize to `gs_guess` instead of the usual `gs_target`:

In [1]:
from __future__ import print_function
import pygsti
from pygsti.construction import std1Q_XYI

In [2]:
#Generate some fake data (all usual stuff here)
gs_target = std1Q_XYI.gs_target.copy()
gs_datagen = std1Q_XYI.gs_target.depolarize(gate_noise=0.01, spam_noise=0.001).rotate( (0,0,0.05) )
gs_datagen['Gx'].depolarize(0.1) #depolarize Gx even further
listOfExperiments = pygsti.construction.make_lsgst_experiment_list(
    gs_target, std1Q_XYI.fiducials, std1Q_XYI.fiducials, std1Q_XYI.germs, [1,2,4])
ds = pygsti.construction.generate_fake_data(gs_datagen, listOfExperiments, nSamples=1000,
                                            sampleError="binomial", seed=1234)
gs_target.set_all_parameterizations("TP") #we'll do TP-constrained GST below

#### Create a "guess" gateset that anticipates a more-depolarized Gx gate

In [3]:
gs_guess = std1Q_XYI.gs_target.copy()
gs_guess['Gx'].depolarize(0.1)

#### Run GST with and without the guess gate set

In [4]:
# GST with standard "ideal target" gauge optimization
results1 = pygsti.do_long_sequence_gst(
    ds, gs_target, std1Q_XYI.fiducials, std1Q_XYI.fiducials, std1Q_XYI.germs, [1,2,4],
    gaugeOptParams={'itemWeights': {'gates': 1, 'spam': 1}}, verbosity=1)

--- Gate Sequence Creation ---
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  450 gate strings ---
Iterative MLGST Total Time: 1.9s


In [5]:
# GST with our guess as the gauge optimization target (just add "targetGateset" to gaugeOptParams)
results2 = pygsti.do_long_sequence_gst(
    ds, gs_target, std1Q_XYI.fiducials, std1Q_XYI.fiducials, std1Q_XYI.germs, [1,2,4],
    gaugeOptParams={'targetGateset': gs_guess, 
                    'itemWeights': {'gates': 1, 'spam': 1}}, verbosity=1)

--- Gate Sequence Creation ---
--- LGST ---
--- Iterative MLGST: [##################################################] 100.0%  450 gate strings ---
Iterative MLGST Total Time: 1.5s


In [6]:
#Note: you can also use the gaugeOptTarget
results3 = pygsti.do_stdpractice_gst(
    ds, gs_target, std1Q_XYI.fiducials, std1Q_XYI.fiducials, std1Q_XYI.germs, [1,2,4],
    gaugeOptTarget=gs_guess, verbosity=1)

-- Std Practice:  [##################################################] 100.0%  (Target) --


#### Comparisons
After running both the "ideal-target" and "gs_guess-target" gauge optimizations, we can compare them with the ideal targets and the data-generating gates themselves.  We see that using `gs_guess` results in a similar frobenius distance to the ideal targets, a slightly closer estimate to the data-generating gate set, and reflects our expectation that the `Gx` gate is slightly worse than the other gates.

In [7]:
gs1 = results1.estimates['default'].gatesets['go0']
gs2 = results2.estimates['default'].gatesets['go0']
print("Diff between ideal and ideal-target-gauge-opt = ", gs1.frobeniusdist(gs_target))
print("Diff between ideal and gs_guess-gauge-opt = ", gs2.frobeniusdist(gs_target))
print("Diff between data-gen and ideal-target-gauge-opt = ", gs1.frobeniusdist(gs_datagen))
print("Diff between data-gen and gs_guess-gauge-opt = ", gs2.frobeniusdist(gs_datagen))
print("Diff between ideal-target-GO and gs_guess-GO = ", gs1.frobeniusdist(gs2))

print("\nPer-op difference between ideal and ideal-target-GO")
print(gs1.strdiff(gs_target))

print("\nPer-op difference between ideal and gs_guess-GO")
print(gs2.strdiff(gs_target))

Diff between ideal and ideal-target-gauge-opt =  0.026584717169268274
Diff between ideal and gs_guess-gauge-opt =  0.026585206307709976
Diff between data-gen and ideal-target-gauge-opt =  0.011644828600115311
Diff between data-gen and gs_guess-gauge-opt =  0.011579947839198834
Diff between ideal-target-GO and gs_guess-GO =  0.0001566885057656346

Per-op difference between ideal and ideal-target-GO
Gateset Difference:
 Preps:
  rho0 = 0.0180632
 POVMs:
  Mdefault:     0 = 0.0147348
    1 = 0.0147348
 Gates:
  Gi = 0.0797907
  Gx = 0.186213
  Gy = 0.0245469


Per-op difference between ideal and gs_guess-GO
Gateset Difference:
 Preps:
  rho0 = 0.0179098
 POVMs:
  Mdefault:     0 = 0.0144141
    1 = 0.0144141
 Gates:
  Gi = 0.0797912
  Gx = 0.186273
  Gy = 0.0246119



#### Adding a gauge optimization to existing `Results`
We can also include our `gs_guess` as the `targetGateset` when adding a new gauge-optimized result.  See other examples for more info on using `add_gaugeoptimized`.

In [8]:
results1.estimates['default'].add_gaugeoptimized( {'targetGateset': gs_guess, 
                                                   'itemWeights': {'gates': 1, 'spam': 1}},
                                                label="using gs_guess")

In [9]:
gs1b = results1.estimates['default'].gatesets['using gs_guess']
print(gs1b.frobeniusdist(gs2)) # gs1b is the same as gs2

1.2898012380872714e-14
